<a href="https://colab.research.google.com/github/mauricio-ms/motor-imagery-convolutional-recurrent-neural-network/blob/master/notebooks/cnn-1d-tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Verify if TPU is enabled

In [24]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.3.0


Mount Google Drive directory

In [6]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Unzip dataset

In [7]:
!tar -xf "drive/My Drive/motor-imagery-convolutional-recurrent-neural-network/preprocessed-csv-files.tar.xz"

Install needed packages

In [8]:
!pip install colorlog

In [9]:
import logging
import colorlog


def get_logger(name="__main__", debug=False):
    log_format = (
        "[%(levelname)s] "
        "%(asctime)s - "
        "%(name)s: "
        "%(funcName)s - "
        "%(message)s"
    )
    bold_seq = "\033[1m"
    colorlog_format = (
        f"{bold_seq} "
        "%(log_color)s "
        f"{log_format}"
    )
    colorlog.basicConfig(format=colorlog_format)

    logger = logging.getLogger(name)
    if debug:
        logger.setLevel(logging.DEBUG)
    else:
        logger.setLevel(logging.INFO)

    # Output full log
    fh = logging.FileHandler("app.log")
    fh.setLevel(logging.DEBUG)
    formatter = logging.Formatter(log_format)
    fh.setFormatter(formatter)
    logger.addHandler(fh)

    return logger

In [10]:
import os
import tensorflow as tf
import numpy as np

LOGGER = get_logger("physionet.py")
N_CHANNELS = 64
PREPROCESSED_CSV_FILES_DIR = "preprocessed-csv-files"

In [11]:
def load_data(train_size=0.75, validation_size=None, n_subjects=None, **kwargs):
    LOGGER.info("Loading Physionet dataset ...")
    subjects = np.array(sorted(os.listdir(PREPROCESSED_CSV_FILES_DIR)))
    if n_subjects is not None:
        subjects = subjects[:n_subjects]
    train_subjects, test_subjects = _train_test_split_subjects(subjects, train_size)
    if validation_size is not None:
        train_subjects, validation_subjects = _train_test_split_subjects(train_subjects, 1-validation_size)
        LOGGER.info(f"(Train, Validation, Test) Subjects = "
                    f"({len(train_subjects)}, {len(validation_subjects)}, {len(test_subjects)})")
        return _load_set(train_subjects, **kwargs), \
            _load_set(validation_subjects, **kwargs), \
            _load_set(test_subjects, **kwargs)

    LOGGER.info(f"(Train, Validation) Subjects = ({len(train_subjects)}, {len(test_subjects)})")
    return _load_set(train_subjects, **kwargs), _load_set(test_subjects, **kwargs)


def _train_test_split_subjects(subjects, train_size):
    train_subjects_mask = np.random.rand(len(subjects)) < train_size
    return subjects[train_subjects_mask], subjects[~train_subjects_mask]


def _load_set(subjects, n_readers=5, n_parse_threads=5, batch_size=100,
              convert_to_2d=False, expand_dim=False):
    path_files = [os.path.join(PREPROCESSED_CSV_FILES_DIR, subject, file_name)
                  for subject in subjects
                  for file_name in sorted(os.listdir(os.path.join(PREPROCESSED_CSV_FILES_DIR, subject)))]
    dataset = tf.data.Dataset.list_files(path_files)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers)
    dataset = dataset.map(lambda r: _preprocess(r, convert_to_2d=convert_to_2d,
                                                expand_dim=expand_dim),
                          num_parallel_calls=n_parse_threads)
    return dataset.batch(batch_size).prefetch(1)


def _preprocess(eeg_record, convert_to_2d=False, expand_dim=False):
    # Create the definitions for the columns (channels + label)
    # The empty array tells TensorFlow to raise exception to missing values
    defs = [tf.constant([], dtype=tf.float32)] * (N_CHANNELS + 1)
    fields = tf.io.decode_csv(eeg_record, record_defaults=defs)
    x = _get_features(fields, convert_to_2d=convert_to_2d)
    if expand_dim:
        x = x[..., np.newaxis]
    y = tf.cast(tf.stack(fields[-1:]), tf.int32)
    return x, y


def _get_features(fields, convert_to_2d=False):
    return tf.stack(fields[:-1])

In [ ]:
from tensorflow import keras

train_set, validation_set, test_set = load_data(expand_dim=True, 
                                                validation_size=0.20,
                                                n_readers=20,
                                                n_parse_threads=20)

In [32]:
# This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = "grpc://" + os.environ["COLAB_TPU_ADDR"]
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_WORKER)
tf.config.experimental_connect_to_host(resolver.master())
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

  [WARNING] 2020-08-13 02:47:06,309 - tensorflow: initialize_tpu_system - TPU system grpc://10.22.229.194:8470 has already been initialized. Reinitializing the TPU can cause previously created variables on TPU to be lost.


INFO:tensorflow:Initializing the TPU system: grpc://10.22.229.194:8470


  [INFO] 2020-08-13 02:47:06,315 - tensorflow: initialize_tpu_system - Initializing the TPU system: grpc://10.22.229.194:8470


INFO:tensorflow:Clearing out eager caches


  [INFO] 2020-08-13 02:47:25,982 - tensorflow: initialize_tpu_system - Clearing out eager caches


...

  [INFO] 2020-08-13 02:47:26,038 - tensorflow: _query_tpu_system_metadata - *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


  [INFO] 2020-08-13 02:47:26,040 - tensorflow: _query_tpu_system_metadata - *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


  [INFO] 2020-08-13 02:47:26,043 - tensorflow: _query_tpu_system_metadata - *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [29]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

strategy = tf.distribute.experimental.TPUStrategy(tpu)
print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  ['10.22.229.194:8470']


  [WARNING] 2020-08-13 02:44:05,922 - tensorflow: initialize_tpu_system - TPU system grpc://10.22.229.194:8470 has already been initialized. Reinitializing the TPU can cause previously created variables on TPU to be lost.


INFO:tensorflow:Initializing the TPU system: grpc://10.22.229.194:8470


  [INFO] 2020-08-13 02:44:05,925 - tensorflow: initialize_tpu_system - Initializing the TPU system: grpc://10.22.229.194:8470


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


  [INFO] 2020-08-13 02:44:14,746 - tensorflow: _query_tpu_system_metadata - *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


  [INFO] 2020-08-13 02:44:14,748 - tensorflow: _query_tpu_system_metadata - *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [34]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  ['10.22.229.194:8470']


  [WARNING] 2020-08-13 02:50:44,421 - tensorflow: initialize_tpu_system - TPU system grpc://10.22.229.194:8470 has already been initialized. Reinitializing the TPU can cause previously created variables on TPU to be lost.


INFO:tensorflow:Initializing the TPU system: grpc://10.22.229.194:8470


  [INFO] 2020-08-13 02:50:44,425 - tensorflow: initialize_tpu_system - Initializing the TPU system: grpc://10.22.229.194:8470


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


  [INFO] 2020-08-13 02:50:52,063 - tensorflow: _query_tpu_system_metadata - *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


  [INFO] 2020-08-13 02:50:52,064 - tensorflow: _query_tpu_system_metadata - *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [35]:
with strategy.scope():
  model = keras.models.Sequential([
      keras.layers.Conv1D(32, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME",
                          input_shape=[64, 1]),
      keras.layers.Conv1D(64, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME"),
      keras.layers.Conv1D(128, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME"),
      keras.layers.Flatten(),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1024, activation="relu",
                        kernel_initializer="he_normal"),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(5, activation="softmax")
  ])

  optimizer = keras.optimizers.Adam(lr=1e-4)
  model.compile(loss="sparse_categorical_crossentropy",
                optimizer=optimizer,
                metrics=["accuracy"])

  [INFO] 2020-08-13 02:51:03,820 - physionet.py: load_data - Loading Physionet dataset ...
  [INFO] 2020-08-13 02:51:03,822 - physionet.py: load_data - (Train, Validation, Test) Subjects = (77, 11, 20)


UnimplementedError: ignored

In [28]:
LOGGER.info("Starting training CNN 1D model ...")
model.fit(train_set, epochs=2, validation_data=validation_set)
LOGGER.info("Training CNN 1D model end!")

  [INFO] 2020-08-13 02:43:22,512 - physionet.py: <module> - Starting training CNN 1D model ...


Epoch 1/2


InternalError: ignored